# Flows.jl

In [1]:
using Flows

INFO: Recompiling stale cache file /home/hofi/.julia/lib/v0.4/Flows.ji for module Flows.


## TimeExpression

### TimeVariable

In [2]:
t = TimeVariable("t")

t

In [3]:
typeof(t)

Flows.TimeVariable

In [4]:
@t_vars s r τ

(s,r,τ)

In [5]:
typeof(s)

Flows.TimeVariable

### TimeLinearCombination

In [6]:
ex = TimeLinearCombination(t,1,s,-2,r,3)

t-2s+3r

The internal representation of a `TimeLinearCombination` is a list of (`TimeExpression`, coefficient)-pairs:

In [7]:
ex.terms

3-element Array{Tuple{Flows.TimeExpression,Real},1}:
 (t,1) 
 (s,-2)
 (r,3) 

The operators +, -, * are overloaded. Therefore, the `TimeLinearCombination` `ex` from above can also be defined in the following way:

In [8]:
ex1 = t - 2s + 3r

t-2s+3r

In [9]:
ex == ex1

true

In [10]:
string(ex)

"t-2s+3r"

In [11]:
t-2ex+11r-2(r-s)

-t+6s+3r

In [12]:
ex-ex

t_zero

In [13]:
string(ex-ex)

"t_zero"

### t_zero
There is a predefined zero element `t_zero` which is implemented as an empty `TimeLinearCombination`:

In [14]:
ex-ex == t_zero

true

In [15]:
typeof(t_zero)

Flows.TimeLinearCombination

In [16]:
t_zero.terms

0-element Array{Tuple{Flows.TimeExpression,Real},1}

### coefficient

In [17]:
ex

t-2s+3r

In [18]:
coefficient(ex,s)

-2

### substitute

In [19]:
substitute(ex,s,ex)

-t+4s-3r

In [20]:
substitute(t,s,r)

t

## SpaceExpression
Here 'space' is ot necessarily physical space, but abstract (Banach) space on which functions (i.e. operators) and flows of differential equations are defined.

### SpaceVariable

In [21]:
u = SpaceVariable("u")

u

In [22]:
@x_vars v,w

(v,w)

### AutonomousFunction

In [23]:
@funs F,G,H

(F,G,H)

In [24]:
typeof(G)

Flows.VectorFieldVariable

### AutonomousFunctionExpression

In [25]:
ex = AutonomousFunctionExpression(F,u)

F(u)

In [26]:
ex1 = F(u)

F(u)

In [27]:
string(ex1)

"F(u)"

In [28]:
ex == ex1

true

`AutonomousFunctionExpression` involving differentials:

In [29]:
ex = F(u,v)

F(u,v)

In [30]:
string(ex)

"F(u,v)"

In [31]:
ex = F(u,v,w)

F(u,v,w)

In [32]:
string(ex)

"F(u,v,w)"

### FlowExpression

In [33]:
ex = FlowExpression(F, t, u, 0) # (the last argument 0 indicates that it is a 0th derivative w.r.t. time)

E(F,t,u)

In [34]:
ex1 = E(F,t,u)

E(F,t,u)

In [35]:
ex == ex1

true

In [36]:
string(ex)

"E(F,t,u)"

`FlowExpression` involving differentials:

In [37]:
ex = E(F,t,u,v)

E(F,t,u,v)

In [38]:
string(ex)

"E(F,t,u,v)"

In [39]:
ex = E(F,t,u,v,w)

E(F,t,u,v,w)

In [40]:
string(ex)

"E(F,t,u,v,w)"

### NonAutonomousFunction

In [41]:
@nonautonomous_funs(S)

(S,)

### NonAutonomousFunctionExpression

In [42]:
ex = S(t,u)

S(t,u)

In [43]:
string(ex)

"S(t,u)"

In [44]:
ex = S(2,t,u,v)

S(2,t,u,v)

In [45]:
string(ex)

"S(2,t,u,v)"

### SpaceLinearCombinations

In [46]:
ex = -17E(F,t,u,v,w) + 2u + F(v,w)

F(v,w)+2u-17E(F,t,u,v,w)

In [47]:
ex-ex

x_zero

In [48]:
string(ex)

"F(v,w)+2u-17E(F,t,u,v,w)"

### x_zero

In [49]:
ex-ex == x_zero

true

In [50]:
string(ex-ex)

"x_zero"

### differential

In [51]:
ex = F(G(u)) + E(F,t,u,w)

F(G(u))+E(F,t,u,w)

In [52]:
string(ex)

"F(G(u))+E(F,t,u,w)"

We form the differential of the expression `ex` with respect to the variable `u` and apply it to the expression `H(v)`. Note that the differntial is a linear map which has to be applied to something ("the slots have to be filled").

In [53]:
differential(ex, u, H(v))

F(G(u),G(u,H(v)))+E(F,t,u,w,H(v))

In [54]:
ex = S(1,t-2s,2u+v)

S(1,t-2s,v+2u)

In [55]:
expand(differential(ex, u, w))

2S(1,t-2s,v+2u,w)

### t_derivative

In [56]:
ex = E(F,t-2s,u+E(G,s,v))

E(F,t-2s,E(G,s,v)+u)

In [57]:
t_derivative(ex, s)

-2F(E(F,t-2s,E(G,s,v)+u))+E(F,t-2s,E(G,s,v)+u,G(E(G,s,v)))

In [58]:
ex = S(2t,S(t,v))

S(2t,S(t,v))

In [59]:
t_derivative(ex, t)

S(2t,S(t,v),S(1,t,v))+2S(1,2t,S(t,v))

### expand

In [60]:
ex = E(F,t,u,2v+3w)

E(F,t,u,3w+2v)

In [61]:
Flows.expand(ex)

2E(F,t,u,v)+3E(F,t,u,w)

In [62]:
ex = G(u, v+w, v+w)

G(u,w+v,w+v)

In [63]:
Flows.expand(ex)

2G(u,w,v)+G(u,w,w)+G(u,v,v)

In [64]:
ex = S(t,u,2v+3w)

S(t,u,3w+2v)

In [65]:
expand(ex)

3S(t,u,w)+2S(t,u,v)

### reduce_order

In [66]:
ex = E(F,t,u,F(u))

E(F,t,u,F(u))

In [67]:
reduce_order(ex)

F(E(F,t,u))

In [68]:
ex = E(F,t,u,F(u),v)

E(F,t,u,F(u),v)

In [69]:
reduce_order(ex)

-E(F,t,u,F(u,v))+F(E(F,t,u),E(F,t,u,v))

In [70]:
ex = E(F,t,u,F(u),v, w)

E(F,t,u,F(u),v,w)

In [71]:
reduce_order(ex)

-E(F,t,u,F(u,v),w)-E(F,t,u,F(u,w),v)+F(E(F,t,u),E(F,t,u,v,w))-E(F,t,u,F(u,v,w))+F(E(F,t,u),E(F,t,u,v),E(F,t,u,w))

### substitute

In [72]:
substitute(E(G,t,v), v, E(F,t,u))

E(G,t,E(F,t,u))

Commutator $[F,G]$:

In [73]:
C_FG = F(u,G(u))-G(u,F(u))

-G(u,F(u))+F(u,G(u))

Double commutator $[F,[F,G]]$ by substituting $G\to[F,G]$ in $[F,G]$:

In [74]:
expand(substitute(C_FG, G, C_FG, u))

G(u,F(u,F(u)))-2F(u,G(u,F(u)))-F(u,G(u),F(u))+F(u,F(u,G(u)))+G(u,F(u),F(u))

### commutator

In [84]:
ex=commutator(F,G,u)

-G(u,F(u))+F(u,G(u))

In [85]:
commutator(F,G,H,u)

H(u,G(u,F(u)))+F(u,G(u,H(u))-H(u,G(u)))+H(u,G(u),F(u))-G(u,H(u),F(u))-G(u,H(u,F(u)))

Verify Jacobi identity $[F,[G,H]]+[H,[F,G]]+[G,[H,F]]=0$:

In [86]:
expand(commutator(F,G,H,u)+commutator(G,H,F,u)+commutator(H,F,G,u))

x_zero

In [89]:
ex=expand(differential(ex,u,v))

F(u,G(u),v)-G(u,F(u,v))+F(u,G(u,v))-G(u,F(u),v)

In [90]:
expand(differential(ex,u,w))

F(u,G(u,v),w)+F(u,G(u),v,w)-G(u,F(u,v),w)+F(u,G(u,w),v)+F(u,G(u,w,v))-G(u,F(u,w),v)-G(u,F(u,v,w))-G(u,F(u),v,w)